In [2]:
import tiktoken
import openai
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import WebBaseLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.retrievers  import BM25Retriever
from langchain.retrievers import EnsembleRetriever
from langchain_community.vectorstores import FAISS

USER_AGENT environment variable not set, consider setting it to identify your requests.


### 1. BM25 기반 검색기 만들기
- BM25는 사용자가 검색한 단어가 특정 문서 내에서 많이 등장할수록 (TF가 높을수록), 전체 문서 중 매우 적은 수의 문서에만 나타나는 단어일수록 (IDF가 클수록), 그리고 문서의 길이가 짧을수록, 더 높은 점수를 주어 해당 문서를 사용자가 원하는 검색 결과의 상위에 배치함

In [ ]:
loader = PyPDFLoader('./gsat_170823.pdf')
pages = loader.load_and_split() # pdf 페이지 단위로 쪼깨서 읽어 들임
print(len(pages))

In [ ]:
model_huggingface = HuggingFaceEmbeddings(model_name='BAAI/bge-m3')

In [ ]:
pages[0]

In [ ]:
pages[0].page_content

In [ ]:
pages[1].page_content

### Chunking하자!

In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
texts = text_splitter.split_documents(pages)

In [ ]:
texts

In [ ]:
len(texts)

In [ ]:
len(texts[0].page_content)

### BM-25 검색기

In [11]:
bm25_retriever = BM25Retriever.from_documents(texts)
bm25_retriever.k = 2

In [ ]:
docs = bm25_retriever.get_relevant_documents("삼성전자의 사업 영역은?")

In [ ]:
for i in docs:
    print(i.metadata)
    print(":")
    print(i.page_content.replace('\n',' '))
    print(len(i.page_content.replace('\n',' ')))
    print("*"*30)

### 임베딩 기반 검색기

In [15]:
chroma_vector = Chroma.from_documents(texts, model_huggingface)
chroma_retriever = chroma_vector.as_retriever(search_kwargs={'k':2})

In [ ]:
docs = chroma_retriever.invoke("삼성전자의 사업 영역은?")
for i in docs:
    print(i.metadata)
    print(":")
    print(i.page_content.replace('\n',' '))
    print(len(i.page_content.replace('\n',' ')))
    print("*"*30)

### BM-25 + 임베딩 앙상블 기반 검색기

In [17]:
ensemble_retriever = EnsembleRetriever(
                    retrievers = [bm25_retriever, chroma_retriever], weight = {0.5,0.5})

docs = ensemble_retriever.invoke("삼성전자의 사업 영역은?")

In [ ]:
for i in docs:
    print(i.metadata)
    print(":")
    print(i.page_content.replace('\n',' '))
    print(len(i.page_content.replace('\n',' ')))
    print("*"*30)